# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [29]:
# Entrar a internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

driver = webdriver.Chrome()

# Pegar a cotação do Dólar
driver.get('https://www.google.com/')
driver.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
driver.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = driver.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')   

# Pegar a cotação do Euro
driver.get('https://www.google.com/')
driver.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
driver.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = driver.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')   

# Pegar a cotação do Lastro - Ouro
driver.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = driver.find_element_by_xpath(
    '//*[@id="comercial"]').get_attribute('value').replace(',', '.')

driver.quit()
print(cotacao_dolar, cotacao_euro, cotacao_ouro)

5.2196 6.130002632 294.12


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [28]:
# Importar e atualizar a base de dados
tabela = pd.read_excel('./produtos.xlsx')
display(tabela) 

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [36]:
# Atualizar a cotação
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = round(float(cotacao_dolar), 2)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = round(float(cotacao_euro), 2)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = round(float(cotacao_ouro), 2)

display(tabela)

# Calcular os Preços Base Reais
tabela['Preço Base Reais'] = round(tabela['Preço Base Original'] * tabela['Cotação'], 2)

display(tabela)

# Calcular os Preços Finais
tabela['Preço Final'] = round(tabela['Preço Base Reais'] * tabela['Margem'], 2)

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.22,5219.95,1.40,7307.93
1,Carro Renault,4500.00,Euro,6.13,27585.00,2.00,55170.00
2,Notebook Dell,899.99,Dólar,5.22,4697.95,1.70,7986.52
3,IPhone,799.00,Dólar,5.22,4170.78,1.70,7090.33
4,Carro Fiat,3000.00,Euro,6.13,18390.00,1.90,34941.00
5,Celular Xiaomi,480.48,Dólar,5.22,2508.11,2.00,5016.22
6,Joia 20g,20.00,Ouro,294.12,5882.40,1.15,6764.76


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.22,5219.95,1.40,7307.93
1,Carro Renault,4500.00,Euro,6.13,27585.00,2.00,55170.00
2,Notebook Dell,899.99,Dólar,5.22,4697.95,1.70,7986.52
3,IPhone,799.00,Dólar,5.22,4170.78,1.70,7090.33
4,Carro Fiat,3000.00,Euro,6.13,18390.00,1.90,34941.00
5,Celular Xiaomi,480.48,Dólar,5.22,2508.11,2.00,5016.22
6,Joia 20g,20.00,Ouro,294.12,5882.40,1.15,6764.76


,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.22,5219.95,1.40,7307.93
1,Carro Renault,4500.00,Euro,6.13,27585.00,2.00,55170.00
2,Notebook Dell,899.99,Dólar,5.22,4697.95,1.70,7986.52
3,IPhone,799.00,Dólar,5.22,4170.78,1.70,7090.33
4,Carro Fiat,3000.00,Euro,6.13,18390.00,1.90,34941.00
5,Celular Xiaomi,480.48,Dólar,5.22,2508.11,2.00,5016.22
6,Joia 20g,20.00,Ouro,294.12,5882.40,1.15,6764.76


### Agora vamos exportar a nova base de preços atualizada

In [37]:
# Exportar a base de dados atualizada
tabela.to_excel('./produtos.xlsx', index=False)